### Performance

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.rnn_exps4

perf_df = []

for i, exp in enumerate(exps):

    task = exp.b2a
    perf = task.get_optimal_choice_probability()
    # perf = task.get_reward_probability()

    df = pd.DataFrame(
        dict(
            trial_id=np.arange(perf.size) + 1,
            perf=perf,
            final_perf=perf[-5:].mean(),
            name=exp.sub_name,
            # grp="struc" if task.is_structured else "unstruc",
            grp=exp.tag,
        )
    )
    perf_df.append(df)

perf_df = pd.concat(perf_df, ignore_index=True)
mab_subjects.GroupData().save(perf_df, "rnn_perf_impure_reset")

In [ ]:
reset_freq = np.array([1, 2, 3])  # Every 1, 2, or 3 sessions
reset_idxs = np.cumsum(np.random.choice(reset_freq, size=200 // reset_freq.min()))

epoch = 0
for session_idx in range(400):
    if session_idx in reset_idxs:
        epoch += 1  # Increment epoch for each reset
    print(f"session {session_idx + 1}, session_group {epoch + 1}")
    # Simulate some processing for the session
    # In a real scenario, you would replace this with actual model training or evaluation logic
    # For example, you might call a function to train or evaluate your model here

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mab_colors import colors_2arm_swap
from neuropy import plotting
from statplotannot.plots import SeabornPlotter, fix_legend
from mab_subjects import GroupData

df = GroupData().rnn_perf_impure_reset
# df = perf_df

fig = plotting.Fig(8, 8, num=1, fontsize=10)

names = df["name"].unique()

for i, name in enumerate(names):
    ax = fig.subplot(fig.gs[i])
    val = name.split("_")
    if val[0] == "structured":
        col = "g"
    if val[0] == "unstructured":
        col = "r"

    SeabornPlotter(
        data=df[df["name"] == name], x="trial_id", y="perf", hue="grp", ax=ax
    ).lineplot(palette=[col], errorbar="se", err_kws=dict(ec=None))
    fix_legend(ax=ax, loc="lower right", ncols=2, only_labels=True, fw="bold", fs=10)
    # ax.set_title("LSTM performance over trials")
    ax.set_title(f"{val[0][0]} on {val[-1][0]}")
    ax.set_ylabel("Pr (High)")
    ax.set_xlabel("")

    ax.set_ylim(0.45, 0.9)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mab_colors import colors_2arm_swap
from neuropy import plotting
from statplotannot.plots import SeabornPlotter, fix_legend
from mab_subjects import GroupData

df = GroupData().rnn_perf_impure_reset
max_u_on_u = df[df["grp"] == "u_on_u"]
max_names = df.groupby("grp").max()["name"].unique()
df = df[df["name"].isin(max_names)]

fig = plotting.Fig(6, 4, num=1, fontsize=10)

ax = fig.subplot(fig.gs[0])
SeabornPlotter(data=df, x="trial_id", y="perf", hue="grp", ax=ax).lineplot(
    palette=colors_2arm_swap(), errorbar="se", err_kws=dict(ec=None)
)
fix_legend(ax=ax, loc="lower right", ncols=2, only_labels=True, fw="bold", fs=10)
# ax.set_title("LSTM performance over trials")
ax.set_title(f"{val[0][0]} on {val[-1][0]}")
ax.set_ylabel("Pr (High)")
ax.set_xlabel("")

# ax.set_ylim(0.45, 0.9)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mab_colors import colors_2arm_swap
from neuropy import plotting
from statplotannot.plots import SeabornPlotter, fix_legend
from mab_subjects import GroupData

df = GroupData().rnn_perf_impure_reset
# max_u_on_u = df[df["grp"] == "u_on_u"]
# max_names = df.groupby("grp").max()["name"].unique()
# df = df[df["name"].isin(max_names)]

fig = plotting.Fig(5, 3, num=1, fontsize=10)
ax = fig.subplot(fig.gs[0])

for g, grp in enumerate(["u_on_u", "u_on_s", "s_on_s", "s_on_u"]):
    g_df = df[df["grp"] == grp]
    max_val = g_df["final_perf"].max()
    g_df = g_df[g_df["final_perf"] == max_val]

    sns.lineplot(
        data=g_df,
        x="trial_id",
        y="perf",
        hue="grp",
        ax=ax,
        palette=[colors_2arm_swap()[g]],
        errorbar="se",
        err_kws=dict(ec=None),
    )
    # ax.set_title("LSTM performance over trials")
    # ax.set_title(f"{val[0][0]} on {val[-1][0]}")
    ax.set_ylabel("Pr (High)")
    ax.set_xlabel("")

fix_legend(ax=ax, loc="lower right", ncols=2, only_labels=True, fw="bold", fs=10)
# ax.get_legend().remove()
# ax.set_ylim(0.45, 0.9)

### Switching probability

In [ ]:
from banditpy.analyses import SwitchProb2Arm
import pandas as pd
import mab_subjects
import numpy as np


exps = mab_subjects.rnn_exps4
sp_df = []

for i, exp in enumerate(exps):

    task = exp.b2a
    sp = SwitchProb2Arm(task).by_trial()

    df = pd.DataFrame(
        dict(
            trial_id=np.arange(sp.size) + 1,
            switch_prob=sp,
            name=exp.sub_name,
            grp=exp.tag,
        )
    )
    sp_df.append(df)

sp_df = pd.concat(sp_df, ignore_index=True)
mab_subjects.GroupData().save(sp_df, "rnn_switch_prob_impure_reset")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw
from mab_colors import colors_2arm_swap
from statplotannot.plots import SeabornPlotter, fix_legend

fig = plotting.Fig(4, 2, size=(8.5, 11), num=1, fontsize=10)

sp_df = mab_subjects.GroupData().rnn_switch_prob_impure_reset

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=sp_df, x="trial_id", y="switch_prob", hue="grp", ax=ax)
sns.lineplot(
    palette=colors_2arm_swap(),
    # palette=["#E89317", "#3980ea"],
    errorbar="se",
    # ci=68,
    err_kws={"edgecolor": None, "alpha": 0.1},
    **plot_kw,
)
fix_legend(ax=ax, loc="upper right", ncols=2, only_labels=True, fw="bold", fs=10)

ax.set_title("LSTM switch probability over trials")
# ax.set_ylim(0.15)

### Conditional switching probability

In [ ]:
from banditpy.analyses import SwitchProb2Arm
import pandas as pd
import mab_subjects
import numpy as np

exps = mab_subjects.rnn_exps4

sp_df = []

for i, exp in enumerate(exps):

    task = exp.b2a
    sp, seq = SwitchProb2Arm(task).by_history(n_past=3, history_as_str=True)

    df = pd.DataFrame(
        dict(
            seq=seq,
            switch_prob=sp,
            name=exp.sub_name,
            grp=exp.tag,
        )
    )
    sp_df.append(df)

sp_df = pd.concat(sp_df, ignore_index=True)
mab_subjects.GroupData().save(sp_df, "rnn_cond_switch_prob_impure_reset")

In [ ]:
from neuropy import plotting
import mab_subjects
from statplotannot.plots import SeabornPlotter
from mab_colors import colors_2arm_swap, colors_2arm

df = mab_subjects.GroupData().rnn_cond_switch_prob_impure_reset
df = df[df["grp"].isin(["s_on_u", "s_on_s"])]

fig = plotting.Fig(7, 3, size=(8.5, 11), num=1, fontsize=10)

ax = fig.subplot(fig.gs[0, :2])
SeabornPlotter(
    data=df,
    x="seq",
    y="switch_prob",
    hue="grp",
    hue_order=["s_on_u", "s_on_s"],
    ax=ax,
).barplot(
    dodge=False, palette=colors_2arm(1), alpha=0.6, errorbar="se"
).bootstrap_test()
ax.tick_params(axis="x", rotation=90)

### Performance matrix
- Bins are represented by probability combination

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d

exps = mab_subjects.rnn_exps3

prob_perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=200)

    probs = np.unique(task.probs[task.is_session_start.astype(bool)], axis=0)

    arr = []
    for prob in probs:
        probs_new = np.array([prob, prob[::-1]])
        prob_perf = task.filter_by_probs(probs_new).get_optimal_choice_probability()
        delta_improvement = np.mean(prob_perf[-5:]) - np.mean(prob_perf[:5])
        final_perf = np.mean(prob_perf[-5:])
        middle_perf = np.mean(prob_perf[50:60])
        # prob_perf = gaussian_filter1d(prob_perf, sigma=2)
        arr.append([prob[0], prob[1], middle_perf, final_perf, delta_improvement])
        arr.append([prob[1], prob[0], middle_perf, final_perf, delta_improvement])

    df = pd.DataFrame(
        np.array(arr),
        columns=["prob1", "prob2", "middle_perf", "final_perf", "delta_improvement"],
    )
    df["name"] = exp.sub_name
    df["grp"] = exp.tag

    prob_perf_df.append(df)

prob_perf_df = pd.concat(prob_perf_df, ignore_index=True)
mab_subjects.GroupData().save(prob_perf_df, "rnn_perf_probability_matrix_impure_reset")

In [ ]:
import matplotlib.pyplot as plt
from neuropy.plotting import Fig
import seaborn as sns
from scipy.stats import binned_statistic_2d
from mab_colors import colors_2arm_swap
from statplotannot.plots import fix_legend
import mab_subjects
from statplotannot.plots import xtick_format
import numpy as np

fig = Fig(6, 6, size=(12, 11), num=1, fontsize=10)

mat_df = mab_subjects.GroupData().rnn_perf_probability_matrix_impure_reset
mat_df = prob_perf_df.copy()
equal_prob_bool = mat_df["prob1"] == mat_df["prob2"]
mat_df = mat_df[~equal_prob_bool]

vmin = [0.4, 0.4, 0.1]
vmax = [0.9, 0.9, 0.4]
titles = [
    "\nMiddle performance",
    "\nfinal_perf",
    "\nDelta performance\n (last 5 - first 5 trials)",
]

for c, col_name in enumerate(["middle_perf", "final_perf", "delta_improvement"]):

    for g, grp in enumerate(["u_on_u", "u_on_s", "s_on_s", "s_on_u"]):
        df = mat_df[mat_df["grp"] == grp]
        x, y, values = df["prob1"], df["prob2"], df[col_name]
        bins = np.linspace(0, 0.9, 10) + 0.05
        centers = (bins[:-1] + bins[1:]) / 2
        centers = np.round(centers, 2)

        H, xedges, yedges, _ = binned_statistic_2d(
            x, y, values, statistic=np.nanmean, bins=bins
        )

        ax = fig.subplot(fig.gs[c, g])
        cplot = ax.pcolormesh(
            xedges,
            yedges,
            H.T,
            cmap="plasma",
            vmin=vmin[c],
            vmax=vmax[c],
            shading="auto",
        )
        ax.set_xlabel("Arm 1 prob.")
        ax.set_title(f"{grp} {titles[c]}")
        ax.set_xticks([0.2, 0.4, 0.6, 0.8])
        ax.set_yticks([0.2, 0.4, 0.6, 0.8])
        if g == 0:
            ax.set_ylabel("Arm 2 prob.")
        if g == 3:
            cb = plt.colorbar(
                cplot, ax=ax, label="Performance", shrink=0.7, anchor=(0, 0.9)
            )
            cb.outline.set_visible(False)

    df_correlated = mat_df[(mat_df["prob1"] + mat_df["prob2"]) == 1]

    ax = fig.subplot(fig.gs[c, 4])
    plot_kw = dict(
        data=df_correlated,
        x="prob1",
        y=col_name,
        hue="grp",
        ax=ax,
        palette=colors_2arm_swap(),
    )
    # sns.stripplot(**plot_kw, dodge=True, size=3, alpha=0.4)
    sns.pointplot(
        **plot_kw,
        dodge=True,
        # markers=["o", "s"],
        # linestyles=["-", "--"],
        lw=2,
        errorbar="se",
        alpha=0.6,
    )

    ax.set_xlabel("Arm 1 - Arm 2 prob.")
    xticks = np.array([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])

    ax.set_xticks(np.arange(6), [f"{p1}-{p2}" for p1, p2 in zip(xticks, xticks[::-1])])
    xtick_format(ax, rotation=45)
    # ax.set_xticks(ha="right")
    # ax.axis["bottom"].major_ticklabels.set_ha("right")
    ax.set_title("Performance for \ncorrelated probabilities")
    ax.set_ylabel(titles[c][1:])
    fix_legend(ax, frameon=True)
    ax.legend_.remove()